#PRIMERA APROXIMACION : PIPELINES
#### Esta implementación de transformers por HuggingFace pintaba muy bien.
#### PIPELINE BETO SPANISH, no he conseguido grandes cosas. En English parece que sí funciona bastante bien.
#### Esta implementación en los modelos que he probado en Español, requiere de pytorch, así que lo he ejecutado en Colab.

In [1]:
# Fist install the library and download the models from github
!pip install transformers
!wget https://users.dcc.uchile.cl/~jperez/beto/cased_2M/pytorch_weights.tar.gz 
!wget https://users.dcc.uchile.cl/~jperez/beto/cased_2M/vocab.txt 
!wget https://users.dcc.uchile.cl/~jperez/beto/cased_2M/config.json 
!tar -xzvf pytorch_weights.tar.gz
!mv config.json pytorch/.
!mv vocab.txt pytorch/.

     |████████████████████████████████| 3.8 MB 5.4 MB/s 
     |████████████████████████████████| 67 kB 3.6 MB/s 
     |████████████████████████████████| 6.5 MB 44.7 MB/s 
     |████████████████████████████████| 895 kB 2.2 MB/s 
     |████████████████████████████████| 596 kB 23.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
--2022-03-20 09:53:16--  https://users.dcc.uchile.cl/~jperez/beto/cased_2M/pytorch_weights.tar.gz
Resolving users.dcc.uchile.cl (users.dcc.uchile.cl)... 200.9.99.211, 192.80.24.4
Connecting to users.dcc.uchile.cl (users.dcc.uchile.cl)|200.9.99.211|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 409871727 (391M) [application/x-gzip]
Saving to: ‘pytorch_weights.tar.gz’

pytorch_weights.tar 100%[===================>] 390.88M  7.57MB/s    in 45s     

2022-03-20 09:54:02 (8.62 MB/s) - ‘pytorch_weights.tar.gz’ saved [409871727/409871727]


In [2]:
# import the necessary
import torch
from transformers import BertForMaskedLM, BertTokenizer

In [3]:
from transformers import BertForSequenceClassification

In [4]:
# create the tokenizer and the model
tokenizer = BertTokenizer.from_pretrained("pytorch/", do_lower_case=False)
model = BertForSequenceClassification.from_pretrained("pytorch/")
e = model.eval()

Some weights of the model checkpoint at pytorch/ were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at pytorch/

In [5]:
from transformers import pipeline
classifier = pipeline('sentiment-analysis')

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)


Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/255M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

In [8]:
#En Inglés funciona muy bien
print(classifier('Today is a beautiful day'))
classifier('Today is a bad day')

[{'label': 'POSITIVE', 'score': 0.9998778104782104}]


[{'label': 'NEGATIVE', 'score': 0.999760091304779}]

In [9]:
from transformers import pipeline
classifier = pipeline('sentiment-analysis',
                        model=model, 
                        tokenizer=tokenizer
                    )

In [10]:
classifier('Hoy es un bonito dia')

[{'label': 'LABEL_1', 'score': 0.5399764180183411}]

In [11]:
classifier('Hoy es dia horrible')

[{'label': 'LABEL_1', 'score': 0.539122998714447}]

####You should probably TRAIN this model : este warning indica que tengo que entrenarlo? quizá debido a ese Warning, no está prediciendo excesivamente bien... donde está el preentreno entonces!??

In [13]:
#Otra prueba con el modelo preentrenado en spanish mrm8488/bert-base-spanish-wwm-cased-finetuned-spa-squad2-es

In [14]:
from transformers import pipeline

classifier = pipeline('sentiment-analysis', 
              model="mrm8488/bert-base-spanish-wwm-cased-finetuned-spa-squad2-es", 
              tokenizer="mrm8488/bert-base-spanish-wwm-cased-finetuned-spa-squad2-es")

Some weights of the model checkpoint at mrm8488/bert-base-spanish-wwm-cased-finetuned-spa-squad2-es were not used when initializing BertForSequenceClassification: ['qa_outputs.weight', 'qa_outputs.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at mrm8488/bert-base-spanish-wwm-cased-finetuned-spa-squad2-es and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it

In [15]:
classifier('Hoy es un bonito dia')

[{'label': 'LABEL_1', 'score': 0.7371445894241333}]

In [16]:
classifier('Hoy es dia horrible')

[{'label': 'LABEL_1', 'score': 0.7186225652694702}]

Mismo warning, parece que requiere de preentrenamiento. En tal caso, no me aportan los pipeline

In [ ]:
#Hago una prueba más, con esta librería pysentimiento, referencia en HuggingFace, que es una abstracción sobre pipelines de transformers en español

In [17]:
pip install pysentimiento

     |████████████████████████████████| 312 kB 5.1 MB/s 
     |████████████████████████████████| 175 kB 59.1 MB/s 
     |████████████████████████████████| 212 kB 55.4 MB/s 
     |████████████████████████████████| 134 kB 52.3 MB/s 
     |████████████████████████████████| 1.1 MB 45.7 MB/s 
     |████████████████████████████████| 127 kB 53.8 MB/s 
     |████████████████████████████████| 271 kB 56.3 MB/s 
     |████████████████████████████████| 94 kB 1.8 MB/s 
     |████████████████████████████████| 144 kB 50.7 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=7b420a377a5490d61e5bcc58f9566d851b40c1623f95de9ef19bb74241a7378f
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
Successfully built emoji
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not

In [ ]:
#Para carga dataset desde gcloud
pip install gcsfs

In [32]:
import torch
from pysentimiento import create_analyzer
analyzer = create_analyzer(task="sentiment", lang="es")
analyzer.predict("Qué gran jugador es Messi")

loading file https://huggingface.co/pysentimiento/robertuito-sentiment-analysis/resolve/main/tokenizer.json from cache at /root/.cache/huggingface/transformers/47dd2d3180a6186d30715516321375322e3a84d5e4656762e083091bbb5d5dc4.0843b07596b388e054bae078721182b4846b9e28a7bbf04d7079b274f8613ae3
loading file https://huggingface.co/pysentimiento/robertuito-sentiment-analysis/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/pysentimiento/robertuito-sentiment-analysis/resolve/main/special_tokens_map.json from cache at /root/.cache/huggingface/transformers/25e0e805456d2786a12b70b86278c6e839d19958cb4f541ee1f78621140098f7.0dc5b1041f62041ebbd23b1297f2f573769d5c97d8b7c28180ec86b8f6185aa8
loading file https://huggingface.co/pysentimiento/robertuito-sentiment-analysis/resolve/main/tokenizer_config.json from cache at /root/.cache/huggingface/transformers/4fdf9a5a8e0a6023e1a9cdef62921158c3db9545e73f8dc0f46340a21bbb64d5.50a2bcf7668df2ff5a82b7b0455533bb4c0db21e6e33565fa

AnalyzerOutput(output=POS, probas={POS: 0.994, NEG: 0.003, NEU: 0.003})

In [33]:
import pandas as pd
df_mini = pd.read_csv('gs://tfm_aideas_datasets/texto_limpio.csv', index_col = 'Unnamed: 0')
df_mini

,ticker,date,body,r_adj,label,texto_limpio
6,ANA,2021-01-07 08:29:03.000,"El Ibex 35 ha tocado por momentos los 8.400, n...",0.050625,1,ibex 35 tocado momentos niveles veía marzo ini...
7,ANA,2021-01-07 10:28:03.000,"El presidente de Estados Unidos, Joe Biden, se...",0.012283,1,presidente joe hecho control dos cámaras congr...
8,ANA,2021-01-07 12:21:00.000,"MADRID, 7 Ene. (EUROPA PRESS) - El Ibex 35 caí...",0.012283,1,7 ibex 35 caía media sesión llevaba selectivo ...
9,ANA,2021-01-07 14:35:00.000,"HUELVA, 7 Ene. (EUROPA PRESS) - Las obras de a...",0.012283,1,7 obras adecuación desdoble carretera acceso m...
10,ANA,2021-01-07 15:28:06.000,El asalto al capitolio no ha asus...,0.012283,1,asalto capitolio asustado principales índices ...
...,...,...,...,...,...,...
53473,TEF,2021-12-30 06:13:41.000,"“Todos tendrán su baja, tal y como se ha hecho...",-0.019280,-1,tal hecho principio mensaje tranquilizador dab...
53474,TEF,2021-12-30 08:27:05.000,La última sesión del año en la Bolsa española ...,-0.019280,-1,última sesión año bolsa española transcurrido ...
53475,TEF,2021-12-30 08:31:09.000,17:46La Comunidad Valenciana notifica 10.889 n...,-0.019280,-1,comunidad valenciana notifica nuevos contagios...
53476,TEF,2021-12-30 08:52:26.000,19:12Compromís pide a Ximo Puig «evaluar todas...,-0.019280,-1,pide ximo puig todas posibles frente coronavir...


In [34]:
# result = analyzer.predict(df_mini.texto_limpio)
# result = analyzer.predict(df_mini.texto_limpio[0:100]) 
result = analyzer.predict(df_mini.body[0:100]) 

  0%|          | 0/4 [00:00<?, ?ba/s]

The following columns in the test set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 100
  Batch size = 32


In [35]:
def get_label_num(x):
  if (x == 'POS'):
    return 1
  elif (x == 'NEG'):
    return -1
  else:
    return 0

In [37]:
label_num_result = [get_label_num(x.output) for x in result]

In [38]:
result = pd.DataFrame({'body':df_mini.body[0:100],
                       'label_num_transformer':label_num_result, 
                       'accuracy':df_mini.label[0:100]})
result

,body,label_num_transformer,accuracy
6,"El Ibex 35 ha tocado por momentos los 8.400, n...",0,1
7,"El presidente de Estados Unidos, Joe Biden, se...",-1,1
8,"MADRID, 7 Ene. (EUROPA PRESS) - El Ibex 35 caí...",-1,1
9,"HUELVA, 7 Ene. (EUROPA PRESS) - Las obras de a...",0,1
10,El asalto al capitolio no ha asus...,-1,1
...,...,...,...
201,Las subastas de renovables vuelven a entrar ho...,0,-1
202,"MADRID, 26 Ene. (EUROPA PRESS) - El Ibex 35 ha...",0,-1
203,"MADRID, 26 Ene. (EUROPA PRESS) - Compromís con...",-1,-1
204,Los títulos del grupo energético repuntan un 1...,0,-1


In [39]:
result.iloc[1,:].body

'El presidente de Estados Unidos, Joe Biden, se ha hecho con el control de las dos cámaras del Congreso tras ganar ayer las elecciones al Senado en Georgia. Se allana así el camino para que la nueva Administración de Estados Unidos -Biden tomará el relevo de Donald Trump previsiblemente el 20 de enero- adopte estímulos económicos adicionales. Las perspectivas de un mayor paquete fiscal de ayudas, más regulación corporativa y más impuestos para compañías y grandes fortunas favorecen a algunos sectores bursátiles y perjudican a otros. En general, los expertos señalan a los valores cíclicos, de energía verde, financieras, biotecnológicas y de infraestructuras como los grandes beneficiados de la \'ola azul\', mientras que defensa, farmacia y tecnología serían los perdedores. Los inversores esperan que el control del Senado obtenido por Biden le permita sacar adelante un mayor gasto en infraestructuras para impulsar a la primera economía mundial, devastada por el Covid-19. El programa de in

In [40]:
result.iloc[4,:].body

'El              asalto al capitolio no ha asustado a los inversores, y los principales índices bursátiles europeos arrancaron la jornada en positivo. También en el Ibex aunque a esta hora se desinfla en negativo con la caída en picado de los valores turísticos por la incertidumbre que generan los posibles anuncios de nuevos confinamientos. Pero en líneas generales, la confirmación de la victoria demócrata en el Senado de los Estados Unidos ha impulsado los mercados, que temían un empate entre demócratas y republicanos con la consecuencia del bloqueo de nuevos estímulos económicos. «Es tal vez el escenario ideal. Una mayoría de 50/50 +1 en el Senado podría ser justo lo que receta el médico: suficiente para sacar adelante más estímulos, pero no lo suficiente para conseguir que se hagan las subidas de impuestos», explica ack Janasiewicz, portfolio manager en Nativis Invetment Managers. Así, con el control del Congreso y el Senado en manos de los demócratas, los expertos explican que los 

In [ ]:
###La prediccion del sentimiento financiero, no parece excesivamente buena

#SEGUNDA APROXIMACION : BERT (clase) español

In [41]:
from __future__ import print_function
import numpy as np

import tensorflow as tf
from tensorflow import keras

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Concatenate, Input, Dense, Dropout, Embedding, LSTM, Bidirectional, SimpleRNN, Flatten, Conv1D, GlobalMaxPooling1D, Activation

In [42]:
pip install tensorflow_text

     |████████████████████████████████| 4.9 MB 5.3 MB/s 
     |████████████████████████████████| 462 kB 60.9 MB/s 


In [43]:
pip install tensorflow_hub

In [44]:
pip install tf-models-official

     |████████████████████████████████| 2.2 MB 5.1 MB/s 
     |████████████████████████████████| 237 kB 33.2 MB/s 
     |████████████████████████████████| 636 kB 52.1 MB/s 
     |████████████████████████████████| 47.8 MB 69 kB/s 
     |████████████████████████████████| 43 kB 1.7 MB/s 
     |████████████████████████████████| 1.2 MB 45.7 MB/s 
     |████████████████████████████████| 352 kB 33.3 MB/s 
     |████████████████████████████████| 1.1 MB 48.8 MB/s 
     |████████████████████████████████| 90 kB 8.2 MB/s 
     |████████████████████████████████| 99 kB 8.2 MB/s 
  Created wheel for py-cpuinfo: filename=py_cpuinfo-8.0.0-py3-none-any.whl size=22257 sha256=03da5cce702b6fbdaea1eae516a0606393d44e2717cd8f7c673043001e0b84ed
  Stored in directory: /root/.cache/pip/wheels/d2/f1/1f/041add21dc9c4220157f1bd2bd6afe1f1a49524c3396b94401
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=4534993c112c3f7b44bb085e6c833faa08385d784f6fc177c9452910c27fbcc1
  Stored in

In [45]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [46]:
from official.nlp import optimization  # to create AdamW optmizer

In [57]:
#@title Buscando un BERT model en español para fine-tune

bert_model_name = 'small_bert/bert_en_uncased_L-12_H-768_A-12'

map_name_to_handle = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-768_A-12/1',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_base/2',
    'electra_small':
        'https://tfhub.dev/google/electra_small/2',
    'electra_base':
        'https://tfhub.dev/google/electra_base/2',
    'experts_pubmed':
        'https://tfhub.dev/google/experts/bert/pubmed/2',
    'experts_wiki_books':
        'https://tfhub.dev/google/experts/bert/wiki_books/2',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/talkheads_ggelu_bert_en_base/1',
}

map_model_to_preprocess = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/1',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_preprocess/1',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/1',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/1',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/1',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/1',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/1',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/1',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/1',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/1',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/1',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/1',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/1',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/1',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/1',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/1',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/1',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/1',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/1',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/1',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/1',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/1',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/1',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/1',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/1',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/1',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/1',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_preprocess/1',
    'electra_small':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/1',
    'electra_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/1',
    'experts_pubmed':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/1',
    'experts_wiki_books':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/1',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/1',
}

tfhub_handle_encoder = map_name_to_handle[bert_model_name]
tfhub_handle_preprocess = map_model_to_preprocess[bert_model_name]

In [62]:
def build_classifier_model(encoder, processer, epochs_set, initial_learning_rate):
    # Definición de la entrada del modelo
    text_input = tf.keras.layers.Input(
        shape=(),
        dtype=tf.string,
        name='text'
    )
    # Definición del preprocesador para bert
    preprocessing_layer = hub.KerasLayer(
        processer,
        name='preprocessing'
    )
    encoder_inputs = preprocessing_layer(text_input)

    # Definición del Modelo Bert para codificar la información.
    encoder = hub.KerasLayer(
        encoder,
        trainable=False,
        name='BERT_encoder'
    )
    outputs = encoder(encoder_inputs)
   
    # Seleccionamos la representación del documento entero.
    net = outputs['pooled_output']
   
    # Definimos la arquitectura del modelo de salida ajustada a la tarea
    net = tf.keras.layers.Dropout(0.5)(net)
    net = tf.keras.layers.Dense(units = 512, activation='selu')(net)
    net = tf.keras.layers.Dense(units = 256, activation='selu')(net)
    net = tf.keras.layers.Dense(units = 128, activation='selu')(net)
    net = tf.keras.layers.Dense(units = 64, activation='selu')(net)
    net = tf.keras.layers.Dense(units = 32, activation='selu')(net)
    net = tf.keras.layers.Dense(units = 16, activation='selu')(net)
    net = tf.keras.layers.Dense(8, activation='selu')(net)  
    # net = tf.keras.layers.Dense(3, activation='softmax', name='classifier')(net)
    net = tf.keras.layers.Dense(1, activation='sigmoid', name='classifier')(net)
    
    classifier_model = tf.keras.Model(text_input, net)
    
    # Definición del optimizador para el problema
    epochs = epochs_set
    steps_per_epoch = df_mini.shape[0]
    num_train_steps = steps_per_epoch * epochs
    num_warmup_steps = int(0.1*num_train_steps)
   
    optimizer = optimization.create_optimizer(
        init_lr=initial_learning_rate,
        num_train_steps=num_train_steps,
        num_warmup_steps=num_warmup_steps,
        optimizer_type='adamw'
      )
    # Compilación del modelo
    classifier_model.compile(
        optimizer=optimizer,
        loss="binary_crossentropy",
        metrics=["accuracy"]
      )

    
    return classifier_model 

In [63]:
epocas = 500

model = build_classifier_model(
    tfhub_handle_encoder,
    tfhub_handle_preprocess,
    epocas,
    5e-5
)
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 preprocessing (KerasLayer)     {'input_mask': (Non  0           ['text[0][0]']                   
                                e, 128),                                                          
                                 'input_word_ids':                                                
                                (None, 128),                                                      
                                 'input_type_ids':                                                
                                (None, 128)}                                                

In [64]:
from sklearn.preprocessing import LabelEncoder

lb = LabelEncoder()
labels = lb.fit_transform(df_mini.label)

In [65]:
checkpoint_filepath = '/content/drive/My Drive/Colab Notebooks/bme_clase4/bme_clase1/notebooks/spanish/session4/models_checkpoints/check'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

history = model.fit(x=df_mini.body[:100],
                    y=labels[:100],
                    validation_split=0.1,
                    batch_size=8,
                    epochs=500, callbacks=[model_checkpoint_callback]
                    )

Epoch 1/500
 6/12 [==============>...............] - ETA: 21s - loss: 0.9298 - accuracy: 0.4792

KeyboardInterrupt: ignored

In [ ]:
checkpoint_filepath = '/content/drive/My Drive/Colab Notebooks/bme_clase4/bme_clase1/notebooks/spanish/session4/models_checkpoints/check'

model.load_weights(checkpoint_filepath)

In [ ]:
preds = model.predict(df_mini.text[100:200])